#### Data Preprocessing

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

print("🔧 COMPLETE DATA PREPROCESSING PIPELINE")
print("=" * 50)
print("Goal: Transform ALL features to numerical format for ML")
print("=" * 50)

🔧 COMPLETE DATA PREPROCESSING PIPELINE
Goal: Transform ALL features to numerical format for ML


#### STEP 1: LOAD AND INSPECT RAW DATA

In [31]:
print("\n📂 STEP 1: LOADING RAW DATA")
print("=" * 30)

# Load the original dataset
df = pd.read_csv('../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv')

print(f"✅ Dataset loaded: {df.shape}")
print(f"Rows: {df.shape[0]:,} customers")
print(f"Columns: {df.shape[1]} features")

print("\n📊 Original data types:")
print(df.dtypes)

print("\n🔍 Original categorical features:")
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"Text columns: {categorical_cols}")


📂 STEP 1: LOADING RAW DATA
✅ Dataset loaded: (7043, 21)
Rows: 7,043 customers
Columns: 21 features

📊 Original data types:
customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

🔍 Original categorical features:
Text columns: ['customerID', 'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'Paperles

#### STEP 2: FIX DATA TYPES

In [32]:
print("\n🔧 STEP 2: FIXING DATA TYPES")
print("=" * 30)

# Fix TotalCharges (stored as string, should be numeric)
print("Before TotalCharges conversion:")
print(f"  Data type: {df['TotalCharges'].dtype}")
print(f"  Sample values: {df['TotalCharges'].head().tolist()}")

# Check for problematic values
problematic = df[df['TotalCharges'].str.contains(' ', na=False)]
print(f"  Rows with spaces: {len(problematic)}")

# Convert to numeric
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
print(f"✅ TotalCharges converted to: {df['TotalCharges'].dtype}")
print(f"✅ Missing values created: {df['TotalCharges'].isnull().sum()}")


🔧 STEP 2: FIXING DATA TYPES
Before TotalCharges conversion:
  Data type: object
  Sample values: ['29.85', '1889.5', '108.15', '1840.75', '151.65']
  Rows with spaces: 11
✅ TotalCharges converted to: float64
✅ Missing values created: 11


#### STEP 3: HANDLE MISSING VALUES

In [33]:
print("\n🩹 STEP 3: HANDLING MISSING VALUES")
print("=" * 35)

# Check missing values
missing_summary = df.isnull().sum()
print("Missing values per column:")
for col, missing_count in missing_summary.items():
    if missing_count > 0:
        print(f"  {col}: {missing_count} ({missing_count/len(df)*100:.1f}%)")

# Handle TotalCharges missing values with business logic
if df['TotalCharges'].isnull().sum() > 0:
    # New customers (tenure = 0) should have TotalCharges = 0
    new_customer_mask = (df['TotalCharges'].isnull()) & (df['tenure'] == 0)
    df.loc[new_customer_mask, 'TotalCharges'] = 0
    print(f"✅ Set TotalCharges=0 for {new_customer_mask.sum()} new customers")
    
    # Fill remaining with median
    remaining_missing = df['TotalCharges'].isnull().sum()
    if remaining_missing > 0:
        median_val = df['TotalCharges'].median()
        df['TotalCharges'].fillna(median_val, inplace=True)
        print(f"✅ Filled {remaining_missing} values with median: {median_val:.2f}")

print(f"✅ Final missing values: {df.isnull().sum().sum()}")


🩹 STEP 3: HANDLING MISSING VALUES
Missing values per column:
  TotalCharges: 11 (0.2%)
✅ Set TotalCharges=0 for 11 new customers
✅ Final missing values: 0


#### STEP 4: FEATURE ENGINEERING

In [34]:
print("\n🏗️ STEP 4: FEATURE ENGINEERING")
print("=" * 35)

# Create meaningful business features
df['Customer_Lifetime_Value'] = df['MonthlyCharges'] * df['tenure']
df['Avg_Monthly_Spend'] = df['TotalCharges'] / (df['tenure'] + 1)

# Count services (all the Yes/No service columns)
service_columns = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                   'TechSupport', 'StreamingTV', 'StreamingMovies']
df['Services_Count'] = 0
for col in service_columns:
    df['Services_Count'] += (df[col] == 'Yes').astype(int)

df['Price_Per_Service'] = df['MonthlyCharges'] / (df['Services_Count'] + 1)

# Tenure groups
df['Tenure_Group'] = pd.cut(df['tenure'], 
                           bins=[0, 12, 24, 60, 100], 
                           labels=['New', 'Growing', 'Established', 'Loyal'])

print("✅ Created 5 new features:")
new_features = ['Customer_Lifetime_Value', 'Avg_Monthly_Spend', 'Services_Count', 
                'Price_Per_Service', 'Tenure_Group']
for feature in new_features:
    print(f"  - {feature}")


🏗️ STEP 4: FEATURE ENGINEERING
✅ Created 5 new features:
  - Customer_Lifetime_Value
  - Avg_Monthly_Spend
  - Services_Count
  - Price_Per_Service
  - Tenure_Group


#### STEP 5: COMPREHENSIVE CATEGORICAL ENCODING

In [35]:
print("\n🔄 STEP 5: CATEGORICAL ENCODING (COMPLETE)")
print("=" * 45)

# Make a copy for encoding
df_encoded = df.copy()

# 1. BINARY FEATURES (Yes/No) - Convert to 1/0
binary_features = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
print("\n📍 Binary encoding (Yes=1, No=0):")
for feature in binary_features:
    df_encoded[feature + '_encoded'] = df_encoded[feature].map({'Yes': 1, 'No': 0})
    print(f"  ✅ {feature} → {feature}_encoded")

# 2. SERVICE FEATURES (Yes/No/No internet service) - Convert to 1/0
service_features = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                   'TechSupport', 'StreamingTV', 'StreamingMovies']
print("\n📍 Service encoding (Yes=1, No/No service=0):")
for feature in service_features:
    df_encoded[feature + '_encoded'] = df_encoded[feature].map({
        'Yes': 1, 
        'No': 0, 
        'No internet service': 0
    })
    print(f"  ✅ {feature} → {feature}_encoded")

# 3. MULTIPLE LINES (special case)
print("\n📍 MultipleLines encoding:")
df_encoded['MultipleLines_encoded'] = df_encoded['MultipleLines'].map({
    'Yes': 1, 
    'No': 0, 
    'No phone service': 0
})
print("  ✅ MultipleLines → MultipleLines_encoded")

# 4. CONTRACT (ordinal - has natural order)
print("\n📍 Contract encoding (ordinal):")
contract_mapping = {'Month-to-month': 0, 'One year': 1, 'Two year': 2}
df_encoded['Contract_encoded'] = df_encoded['Contract'].map(contract_mapping)
print("  ✅ Contract → Contract_encoded (0=Month-to-month, 1=One year, 2=Two year)")

# 5. NOMINAL FEATURES (no natural order) - One-hot encoding
nominal_features = ['gender', 'InternetService', 'PaymentMethod']
print("\n📍 One-hot encoding (no natural order):")
for feature in nominal_features:
    # Create dummy variables
    dummies = pd.get_dummies(df_encoded[feature], prefix=feature)
    df_encoded = pd.concat([df_encoded, dummies], axis=1)
    print(f"  ✅ {feature} → {list(dummies.columns)}")

# 6. TARGET VARIABLE
print("\n📍 Target variable encoding:")
df_encoded['Churn_binary'] = df_encoded['Churn'].map({'No': 0, 'Yes': 1})
print("  ✅ Churn → Churn_binary (No=0, Yes=1)")

print(f"\n✅ Dataset shape after encoding: {df_encoded.shape}")


🔄 STEP 5: CATEGORICAL ENCODING (COMPLETE)

📍 Binary encoding (Yes=1, No=0):
  ✅ Partner → Partner_encoded
  ✅ Dependents → Dependents_encoded
  ✅ PhoneService → PhoneService_encoded
  ✅ PaperlessBilling → PaperlessBilling_encoded

📍 Service encoding (Yes=1, No/No service=0):
  ✅ OnlineSecurity → OnlineSecurity_encoded
  ✅ OnlineBackup → OnlineBackup_encoded
  ✅ DeviceProtection → DeviceProtection_encoded
  ✅ TechSupport → TechSupport_encoded
  ✅ StreamingTV → StreamingTV_encoded
  ✅ StreamingMovies → StreamingMovies_encoded

📍 MultipleLines encoding:
  ✅ MultipleLines → MultipleLines_encoded

📍 Contract encoding (ordinal):
  ✅ Contract → Contract_encoded (0=Month-to-month, 1=One year, 2=Two year)

📍 One-hot encoding (no natural order):
  ✅ gender → ['gender_Female', 'gender_Male']
  ✅ InternetService → ['InternetService_DSL', 'InternetService_Fiber optic', 'InternetService_No']
  ✅ PaymentMethod → ['PaymentMethod_Bank transfer (automatic)', 'PaymentMethod_Credit card (automatic)', 'Pa

#### STEP 6: FEATURE SCALING

In [36]:
print("\n📏 STEP 6: FEATURE SCALING")
print("=" * 25)

# Identify numerical features to scale
numerical_features = ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges',
                     'Customer_Lifetime_Value', 'Avg_Monthly_Spend', 'Services_Count', 
                     'Price_Per_Service']

print("Features to scale:")
for feature in numerical_features:
    if feature in df_encoded.columns:
        print(f"  - {feature}: {df_encoded[feature].min():.2f} to {df_encoded[feature].max():.2f}")

# Apply StandardScaler
scaler = StandardScaler()
df_encoded[numerical_features] = scaler.fit_transform(df_encoded[numerical_features])

print("\n✅ Applied StandardScaler - all features now have mean≈0, std≈1")


📏 STEP 6: FEATURE SCALING
Features to scale:
  - SeniorCitizen: 0.00 to 1.00
  - tenure: 0.00 to 72.00
  - MonthlyCharges: 18.25 to 118.75
  - TotalCharges: 0.00 to 8684.80
  - Customer_Lifetime_Value: 0.00 to 8550.00
  - Avg_Monthly_Spend: 0.00 to 118.97
  - Services_Count: 0.00 to 6.00
  - Price_Per_Service: 8.65 to 77.90

✅ Applied StandardScaler - all features now have mean≈0, std≈1


#### STEP 7: FINAL CLEANUP

In [37]:
print("\n🔪 STEP 7: FINAL CLEANUP")
print("=" * 25)

# Remove original categorical columns (keep only encoded versions)
columns_to_drop = [
    'customerID',  # ID column
    'Churn',       # Original target (keep Churn_binary)
    'Tenure_Group',  # Categorical version (business insights only)
    # Original categorical features (we have encoded versions)
    'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
    'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
    'PaperlessBilling', 'PaymentMethod'
]

# Drop columns that exist
columns_dropped = []
for col in columns_to_drop:
    if col in df_encoded.columns:
        df_encoded.drop(col, axis=1, inplace=True)
        columns_dropped.append(col)

print(f"✅ Dropped {len(columns_dropped)} original categorical columns")
print(f"Dropped: {columns_dropped}")

# Final dataset summary
print(f"\n🎯 FINAL DATASET SUMMARY")
print("=" * 30)
print(f"Shape: {df_encoded.shape}")
print(f"Features: {df_encoded.shape[1] - 1} (excluding target)")
print(f"Target: Churn_binary")

# Check data types
print("\nFinal data types:")
print(df_encoded.dtypes.value_counts())

# Verify no text values remain
text_columns = df_encoded.select_dtypes(include=['object']).columns.tolist()
if len(text_columns) == 0:
    print("✅ SUCCESS: All features are numerical!")
else:
    print(f"⚠️ WARNING: Still have text columns: {text_columns}")

print("\nFinal feature list:")
features = [col for col in df_encoded.columns if col != 'Churn_binary']
for i, feature in enumerate(features, 1):
    print(f"  {i:2d}. {feature}")

print(f"\nTarget variable distribution:")
print(df_encoded['Churn_binary'].value_counts())
churn_rate = df_encoded['Churn_binary'].mean() * 100
print(f"Churn rate: {churn_rate:.1f}%")


🔪 STEP 7: FINAL CLEANUP
✅ Dropped 18 original categorical columns
Dropped: ['customerID', 'Churn', 'Tenure_Group', 'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']

🎯 FINAL DATASET SUMMARY
Shape: (7043, 30)
Features: 29 (excluding target)
Target: Churn_binary

Final data types:
int64      13
bool        9
float64     8
Name: count, dtype: int64
✅ SUCCESS: All features are numerical!

Final feature list:
   1. SeniorCitizen
   2. tenure
   3. MonthlyCharges
   4. TotalCharges
   5. Customer_Lifetime_Value
   6. Avg_Monthly_Spend
   7. Services_Count
   8. Price_Per_Service
   9. Partner_encoded
  10. Dependents_encoded
  11. PhoneService_encoded
  12. PaperlessBilling_encoded
  13. OnlineSecurity_encoded
  14. OnlineBackup_encoded
  15. DeviceProtection_encoded
  16. TechSupport_encoded
  17. Stream

#### STEP 8: SAVE PROCESSED DATA

In [45]:
print("\n💾 STEP 8: SAVING PROCESSED DATA")
print("=" * 35)

# Ensure directory exists
import os
os.makedirs('../data/processed', exist_ok=True)

# Save final processed dataset
output_file = '../data/processed/telco_churn_final_processed.csv'
df_encoded.to_csv(output_file, index=False)
print(f"✅ Saved final dataset: {output_file}")

# Create feature list for reference
feature_list = [col for col in df_encoded.columns if col != 'Churn_binary']
feature_info = pd.DataFrame({
    'Feature': feature_list,
    'Type': ['Engineered' if any(x in col for x in ['Customer_', 'Avg_', 'Services_', 'Price_']) 
             else 'Encoded' if '_encoded' in col or any(x in col for x in ['gender_', 'InternetService_', 'PaymentMethod_'])
             else 'Original' for col in feature_list]
})

feature_info.to_csv('../data/processed/feature_reference.csv', index=False)
print("✅ Saved feature reference: ../data/processed/feature_reference.csv")

print("\n" + "="*50)
print("🎉 PREPROCESSING COMPLETE!")
print("🎯 Ready for Model Development!")
print("="*50)


💾 STEP 8: SAVING PROCESSED DATA
✅ Saved final dataset: ../data/processed/telco_churn_final_processed.csv
✅ Saved feature reference: ../data/processed/feature_reference.csv

🎉 PREPROCESSING COMPLETE!
🎯 Ready for Model Development!


In [46]:
df3 = pd.read_csv(r"C:\Users\varen\OneDrive\Documents\Projects\customer-churn-prediction-ml\data\processed\telco_churn_final_processed.csv")
df3.head(3)

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Customer_Lifetime_Value,Avg_Monthly_Spend,Services_Count,Price_Per_Service,Partner_encoded,Dependents_encoded,...,gender_Female,gender_Male,InternetService_DSL,InternetService_Fiber optic,InternetService_No,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_binary
0,-0.439916,-1.277445,-1.160323,-0.992611,-0.993448,-1.441115,-0.561776,-0.749701,1,0,...,True,False,True,False,False,False,False,True,False,0
1,-0.439916,0.066327,-0.259629,-0.172165,-0.151588,-0.163684,-0.020519,-0.461719,0,0,...,False,True,True,False,False,False,False,False,True,0
2,-0.439916,-1.236724,-0.362660,-0.958066,-0.959071,-0.750249,-0.020519,-0.535045,0,0,...,False,True,True,False,False,False,False,False,True,1


In [47]:
df3.shape

(7043, 30)

In [43]:
# Check data types for confirmation
print("Data types in final dataset:")
print(df_encoded.dtypes.value_counts())


Data types in final dataset:
int64      13
bool        9
float64     8
Name: count, dtype: int64
